# Praca domowa 1- eksploracja danych

In [ ]:
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
data=pd.DataFrame(pd.read_json('https://api.apispreadsheets.com/api/dataset/forest-fires/',orient='split'))

In [ ]:
print(f'liczba obserwacji: {data.shape[0]}, Liczba kolumn: {data.shape[1]}')

## Rozpoczniemy od sprawdzenia kompletności danych.

In [ ]:
data.info()

### Żadna z kolumn nie zawiera nulli więc dane nie są wybrakowane

## Pprzebadania naszego targetu,który chcemy miodelować, czyli daną area.

In [ ]:
data['area'].describe()

## Łatwo zauważyć, że większość obserwacji znajduje się blisko zera.

In [ ]:
print(f'Kurtoza: {data["area"].kurt()}')
print(f'Skośność: {data["area"].skew()}')
sns.distplot(data['area'])

## Z wykresu szybko wnioskujemy, że przydałoby się transformować tą zmienną do jej analizy. Skorzystamy z $\log(x)$ zgodnie z sugestią autorów danych.

In [ ]:
data['area(log)']=data['area'].map(lambda x: np.log(x+1))

In [ ]:
print(f'Kurtoza: {data["area(log)"].kurt()}')
print(f'Skośność: {data["area(log)"].skew()}')
sns.distplot(data['area(log)'])

## Od razu widać większą przejrzystość

In [ ]:
sns.boxplot(x='month',y='area(log)',data=data,order=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],)

## Łatwo zauważyć, że to włąśnie podczas miesięcy letnich miały miejsce najgorsze pożary, co jednak ciekawe to w maju i w grudniu dolny kwartyl nie jest w zerze(bądź bardzo blisko niego).

In [ ]:
sns.boxplot(x='day',y='area(log)',data=data,order=['mon','tue','wed','thu','fri','sat','sun'])

In [ ]:
sns.countplot(data=data,x='day',order=['mon','tue','wed','thu','fri','sat','sun'])

## Tu bez większych niespodzianek dzień tygodnia nie wpływa zbytnio na rozmiar, ani na częstotliwość pożarów pożarów.

## Następnie spojrzymy na cechy ciągłe w poszukiwaniu większej ilości zależności

In [ ]:
data.describe()

In [ ]:
data.hist(bins=10, figsize=(15,10))

## Co ciekawe partie lasu najczęściej dotykane przez pożary są przedzielone pasmami, w których pożary nie występuję(z rozkładu zmiennych X i Y). Widzimy również, że FFMC oraz rain są skupione blisko jednej wartości. A ISI i temp najbardziej przypominają rozkłąd normalny.

## Sprawdźmy więc, w których częściach lasu występują najgroźniejsze pożary.


In [ ]:
df=data.iloc[:,[0,1,12]]


plt.show()
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.bar3d(df['Y'], df['X'],0,0.5,0.5,df['area'], cmap=plt.cm.viridis, linewidth=0.02)
ax.view_init(90, 0)


plt.show()
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.bar3d(df['Y'], df['X'], 0,0.5,0.5,df['area'], cmap=plt.cm.viridis, linewidth=0.02)

plt.show()


## Z wykresów widzimy, że najgorsze pożary wybuchają w centralnej części lasu oraz na obrzeżach.

In [ ]:
data_subset=data[['FFMC','rain','ISI','temp','area','area(log)']]
sns.set()
sns.pairplot(data_subset, height=5)

## Zauważamy pewne zależności pomiędzy ISI,a FFMC oraz pomiędzy temp, a FFMC.
## Sprawdzamy, więc współczynniki korelacji pomiędzy zmiennymi.

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(),annot=True)

## Zauważamy, że X i Y są ze sobą dość mocno skorelowane podobnie jak zmienne FFMC, DMC, DC, ISI oraz temp. Niestety nie znajdujemy żadnych korelacji pomiędzy area, a innymi zmiennymi.

In [ ]:
plt.figure(figsize=(10,10))
k = 5
cols = data.corr().nlargest(k, 'area')['area'].index
cm = np.corrcoef(data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

## Zmiennymi najbardziej skorelowanymi do area okazują się temp, DMC i X.

In [ ]:
pandas_profiling.ProfileReport(data)

## Automatyczny sposób eksploracji danych pozwala szybko zapoznać się ze strukturą danych, jednak nie daje ich pełnej analizy. Jest to dobry punkt początkowy ukazujący podstawowe zależności między danymi. Nie daje on jednak zbytniej sfobody co do zależności, które wyświetla np. nie możemy sprawdzić korelacji zmiennych ciągłych z kategorycznymi. 